In [1]:
# import my helper functions
import sys; sys.path.append("/exp/sbnd/data/users/lynnt/xsection/")
import nueana as nue

%cd /exp/sbnd/app/users/lynnt/cafpyana
from makedf.makedf import *
from pyanalib.pandas_helpers import *

/exp/sbnd/app/users/lynnt/cafpyana


In [2]:
mcmc_file = "/exp/sbnd/data/users/lynnt/xsection/samples/examples/mc_systs.df"
data_file = "/exp/sbnd/data/users/lynnt/xsection/samples/examples/data.df"

keys = ['nuecc','hdr']

mcmc_dfs = nue.load_dfs(mcmc_file,keys)
data_dfs = nue.load_dfs(data_file,keys)

mcmc_df = mcmc_dfs['nuecc']
data_df = data_dfs['nuecc']

# apply some corrections 
data_df['slc','barycenterFM','flashTime','','',''] = data_df.slc.barycenterFM.flashTime + data_df.frameApplyAtCaf/1e3
mcmc_df['slc','barycenterFM','flashPEs','','',''] = mcmc_df.slc.barycenterFM.flashPEs*0.68

In [3]:
def InSpill(df):
    spill_start = 0.25 
    spill_end = 2.25
    return (df.slc.barycenterFM.flashTime > spill_start) & (df.slc.barycenterFM.flashTime < spill_end)

def InScore(df):
    return (df.slc.barycenterFM.score > 0.02)

def select(indf):
    df_dict = {}
    df = indf.copy()
    # ** these cuts done already in makedf
    # * require nuscore > 0.5
    # * require not clear cosmic 
    # * require reco vertex in AV
    # * require that there is a primary shower (at least one pfp w/ trackScore < 0.5)
    df_dict['preselection'] = df
    
    # * require that the matched (many-to-many) is inside the beam spill
    df = df[InSpill(df) & InScore(df)]
    df_dict['flash matching'] = df

    # * require that primary shower > 500 MeV
    df = df[df.primshw.shw.maxplane_energy > 0.5]
    df_dict['shower energy'] = df 

    # * require track length < 200 cm
    df = df[np.isnan(df.primtrk.trk.len) | (df.primtrk.trk.len < 200)]
    df = df.drop('primtrk',axis=1,level=0)
    df_dict['muon rejection'] = df

    df = df[df.primshw.shw.conversion_gap < 2]
    df_dict['conversion gap'] = df

    df = df[(df.primshw.shw.bestplane_dEdx > 1) & (df.primshw.shw.bestplane_dEdx < 2.5)]
    df_dict['dEdx'] = df

    df = df[df.primshw.shw.open_angle < 0.2]
    df_dict['opening angle'] = df

    return df_dict

In [4]:
mcmc_dict = select(mcmc_df)
data_dict = select(data_df)